# Отчет по практическому заданию №1: Подготовка набора данных
---

Дисциплина: Системы искусственного интеллекта и большие данные

Студент: Платон Евгения Евгеньевна

Группа: КВБО-14-24
  
Преподаватель: Питинов Артем Вадимович



# **1. Цель работы**

Целью данного задания является выполнение основных этапов подготовки набора данных для последующего анализа и построения модели машинного обучения. В рамках работы проведены: загрузка данных, разведочный анализ (EDA), визуализация, предварительная обработка и конструирование/отбор признаков. В качестве источника данных был выбран набор дат «Набор данных об утверждении кредита» с платформой Kaggle, независимой информации о заявках на кредит и возможности по ним.

# **2. Выбор и описание набора данных**

Название набора дат: Набор данных об одобрении кредита

---

Источник: Kaggle

---


Описание: Датасет содержит данные о клиентах, подавших заявки на получение кредита. Включает демографические, финансовые и поведенческие показатели, а также целевую переменную — одобрение кредита.

---


Цель: Проанализировать факторы, влияющие на решение по кредиту, и подготовить данные для построения модели классификации.

# **3. Загрузка данных**
Этап загрузки данных является первым шагом в любом аналитическом процессе. Он заключается в получении доступа к набору данных и его импорте в среду выполнения (в данном случае, Google Colab) в формате, удобном для последующей обработки. Мы использовали библиотеку pandas для работы с табличными данными и kagglehub для прямой загрузки датасета с платформы Kaggle без необходимости скачивать его вручную.

**3.1. Установка зависимостей**

In [ ]:
!pip install kagglehub pandas matplotlib seaborn scikit-learn



**Эта команда устанавливает библиотеки:**

- kagglehub (для загрузки)

- pandas (для работы с данными)

- matplotlib и seaborn (для визуализации)

- scikit-learn (для возможной предобработки и отбора признаков).

**3.2**. **Загрузка датасета**

In [ ]:
import kagglehub
import pandas as pd
import os
import glob

try:
    print("Загружаем датасет...")
    path = kagglehub.dataset_download("anishdevedward/loan-approval-dataset")
    print("Путь к датасету:", path)

    # Поиск CSV-файла
    csv_files = glob.glob(os.path.join(path, "**", "*.csv"), recursive=True)
    if csv_files:
        df = pd.read_csv(csv_files[0])
        print(f"\nЗагружен файл: {csv_files[0]}")
    else:
        print("\nCSV-файлы не найдены.")
        df = None

except Exception as e:
    print(f"Ошибка загрузки: {e}")
    df = None

if df is not None:
    print("\nПервые 5 строк:")
    print(df.head())
    print("\nРазмерность:", df.shape)
    print("\nСтолбцы:", df.columns.tolist())
else:
    print("Данные не загружены.")

Код сначала загружает датасет с Kaggle как ZIP-архив, используя kagglehub. Затем архив распаковывается, и извлечённый CSV-файл читается. Для первичного анализа сразу после загрузки отображаются: несколько первых строк данных, общее количество строк и столбцов, а также перечень названий столбцов. Это позволяет быстро оценить структуру и содержание датасета.

# **4. Разведочный анализ данных (EDA) и визуализация.**
Разведочный анализ позволяет понять основные признаки, показатели пропусков, аномалии и взаимосвязи между переменными.

In [ ]:
if 'df' in locals() and isinstance(df, pd.DataFrame) and df is not None:
    print("\n--- Общая информация ---")
    print(df.info())

    print("\n--- Пропущенные значения ---")
    print(df.isnull().sum())

    print("\n--- Числовые признаки ---")
    print(df.select_dtypes(include='number').describe())

    print("\n--- Категориальные признаки ---")
    print(df.select_dtypes(include='object').describe())

    # Визуализация
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set_style("whitegrid")

    # 1. Распределение целевой переменной
    plt.figure(figsize=(6, 4))
    # Преобразуем bool в int для корректного отображения (0/1)
    sns.countplot(x=df['loan_approved'].astype(int))
    plt.title('Распределение целевой переменной (loan_approved)')
    plt.xlabel('Одобрение кредита (0 = отказано, 1 = одобрено)')
    plt.ylabel('Количество')
    plt.show()

    # 2. Распределение дохода
    plt.figure(figsize=(8, 5))
    sns.histplot(df['income'], bins=30, kde=True, color='skyblue')
    plt.title('Распределение годового дохода')
    plt.xlabel('Доход ($)')
    plt.ylabel('Частота')
    plt.show()

    # 3. Доход и решение по кредиту
    plt.figure(figsize=(8, 5))
    sns.boxplot(data=df, x=df['loan_approved'].astype(int), y='income')
    plt.title('Доход в зависимости от решения по кредиту')
    plt.xlabel('Одобрение кредита (0 = нет, 1 = да)')
    plt.ylabel('Доход ($)')
    plt.show()

    # 4. Кредитный рейтинг и решение
    plt.figure(figsize=(8, 5))
    sns.boxplot(data=df, x=df['loan_approved'].astype(int), y='credit_score')
    plt.title('Кредитный скоринг в зависимости от решения по кредиту')
    plt.xlabel('Одобрение кредита (0 = нет, 1 = да)')
    plt.ylabel('Credit Score')
    plt.show()

    # 5. Матрица корреляции числовых признаков
    plt.figure(figsize=(10, 8))
    numeric_df = df.select_dtypes(include='number')
    corr = numeric_df.corr()
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', linewidths=0.5)
    plt.title('Матрица корреляции числовых признаков')
    plt.show()


    # 6. Визуализация выбросов
    numeric_cols = df.select_dtypes(include='number').columns.tolist()
    if numeric_cols:
        n = len(numeric_cols)
        n_cols = 2
        n_rows = (n + n_cols - 1) // n_cols  # округление вверх

        plt.figure(figsize=(14, 5 * n_rows))
        for i, col in enumerate(numeric_cols, 1):
            plt.subplot(n_rows, n_cols, i)
            sns.boxplot(y=df[col], color='lightcoral')
            plt.title(f'Boxplot: {col}')
        plt.tight_layout()
        plt.show()
    else:
        print("Числовые признаки для анализа выбросов не найдены.")
else:
    print("Данные не загружены — EDA невозможен.")


Гистограмма дохода показывает сильную правостороннюю асимметрию.
Столбчатая диаграмма показывает, что некоторым лицам кредиты одобряются чаще.
Тепловая карта показывает мультиколлинеарность.




# **5. Предварительная обработка данных**

In [ ]:
if 'df' in locals() and isinstance(df, pd.DataFrame) and df is not None:
    print("Пропусков до обработки:", df.isnull().sum().sum())

    # Удаляем строки с пропусками
    df = df.dropna()
    print("Пропусков после обработки:", df.isnull().sum().sum())

    # Удаляем неинформативные категориальные признаки с высокой кардинальностью
    print("\nУдаляем столбцы 'name' и 'city' (высокая кардинальность).")
    df = df.drop(columns=['name', 'city'], errors='ignore')

    # Убеждаемся, что целевая переменная — числовая
    if 'loan_approved' in df.columns:
        df['loan_approved'] = df['loan_approved'].astype(int)
        print("Целевая переменная 'loan_approved' преобразована в int (0/1).")

    print("\nФорма после предобработки:", df.shape)
    print("Оставшиеся столбцы:", df.columns.tolist())
    print("\nПервые 5 строк после обработки:")
    print(df.head())

else:
    print("Данные не загружены — предобработка невозможна.")



Пропущенные значения отсутствуют, но метод универсален.

Категориальные признаки закодированы с помощью One-Hot Encoding.

Параметр уменьшает размерность и излучает зависимости между электродами.

# **6. Конструирование и отбор критериев.**
На этом этапе новые признаки и отбираются наиболее значимые.

In [ ]:
if 'df' in locals() and isinstance(df, pd.DataFrame) and df is not None:
    # Проверяем наличие нужных столбцов до создания нового признака
    if 'loan_amount' in df.columns and 'income' in df.columns:
        # Создаём новый признак: доля кредита от дохода
        df['loan_percent_income'] = df['loan_amount'] / df['income']
        print("Создан новый признак: 'loan_percent_income'")
    else:
        print("Невозможно создать 'loan_percent_income': отсутствуют 'loan_amount' или 'income'")
        print("Доступные столбцы:", df.columns.tolist())

    # Подготовка к отбору признаков
    from sklearn.feature_selection import SelectKBest, f_classif

    target_col = 'loan_approved'
    if target_col not in df.columns:
        print(f"Ошибка: столбец '{target_col}' не найден")
        print("Доступные столбцы:", df.columns.tolist())
    else:
        X = df.drop(columns=[target_col])
        y = df[target_col]

        # Убеждаемся, что все признаки — числовые (SelectKBest требует чисел)
        X_numeric = X.select_dtypes(include=['number'])
        if X_numeric.shape[1] == 0:
            print("Нет числовых признаков для отбора")
        else:
            # Отбор лучших признаков
            k = min(15, X_numeric.shape[1])
            selector = SelectKBest(score_func=f_classif, k=k)
            X_selected = selector.fit_transform(X_numeric, y)

            selected_features = X_numeric.columns[selector.get_support()].tolist()
            print(f"\n Отобрано {len(selected_features)} признаков:")
            print(selected_features)

            # Формируем итог
            df_final = pd.DataFrame(X_selected, columns=selected_features)
            df_final[target_col] = y.values

            print(f"\n Финальная размерность: {df_final.shape}")
            print("Первые 5 строк итогового датасета:")
            print(df_final.head())

            # df_final.to_csv("loan_data_prepared.csv", index=False)
else:
    print("Данные не загружены - отбор признаков невозможен.")



Создан новый индикатор — отношение суммы кредита к доходу.

Использован метод с понижением.

Отобрано 15 наиболее значимых признаков на основе статистической продукции.

# 7. Выводы
В ходе выполнения практического задания №1 был успешно загружен набор данных «Набор данных для одобрения кредита» с платформы Kaggle. Проведен полный цикл подготовки данных: выполнен разведочный анализ с визуализацией распределений и взаимосвязей, обработаны категориальные признаки с помощью One-Hot Encoding, создан новый информативный признак ( ), и осуществлен отбор наиболее релевантных показателей с использованием статистического метода . В результате получен уточненный и структурированный набор данных, готовый к использованию в задачах машинного обучения, в частности — для построения моделей классификации по одобрению кредитов.